# Gensim pakage

In [2]:
## Bag of Words encoring
## TF-IDF encording
## Topic Modeling

In [3]:
!pip install gensim

     |████████████████████████████████| 24.2 MB 452 kB/s eta 0:00:01
     |████████████████████████████████| 103 kB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 24.8 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 6.2 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 11.0 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=409438c5dd09e1394459bdb9e5037c0de2094e1f25739ed544b25e62a7d2dc6d
  Stored in directory: /Users/jhkim/Library/Caches/pip/wheels/bb/1c/9c/412ec03f6d5ac7d41f4b965bde3fc0d1bd201da5ba3e2636de
Successfully built smart-open


## Bag of Words encoring

#### - Dictionary Class use
######     token2id 속성으로 사전저장
######     doc2bow 메서드로 BOW 인코딩
#### - TfidfModel 클래스를 이용하면 TF-IDF 인코딩도 가능

In [4]:
# step 1
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]

In [5]:
# step 2
token_list = [[text for text in doc.split()] for doc in corpus]
token_list

[['This', 'is', 'the', 'first', 'document.'],
 ['This', 'is', 'the', 'second', 'second', 'document.'],
 ['And', 'the', 'third', 'one.'],
 ['Is', 'this', 'the', 'first', 'document?'],
 ['The', 'last', 'document?']]

In [7]:
# step 3 dict. object create
from gensim.corpora import Dictionary

dictionary = Dictionary(token_list)
dictionary.token2id


{'This': 0,
 'document.': 1,
 'first': 2,
 'is': 3,
 'the': 4,
 'second': 5,
 'And': 6,
 'one.': 7,
 'third': 8,
 'Is': 9,
 'document?': 10,
 'this': 11,
 'The': 12,
 'last': 13}

In [9]:
# step 4 BOW ENCORDING
term_matrix = [dictionary.doc2bow(token) for token in token_list]
term_matrix # sparse matrix form

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(0, 1), (1, 1), (3, 1), (4, 1), (5, 2)],
 [(4, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (4, 1), (9, 1), (10, 1), (11, 1)],
 [(10, 1), (12, 1), (13, 1)]]

In [11]:
# step 5 TF-IDF encording
from gensim.models import TfidfModel

tfidf = TfidfModel(term_matrix)

for doc in tfidf[term_matrix]:
    print("doc:")
    for k, v in doc:
        print(k, v) # 중요도 만큼 스케일링(감중)

doc:
0 0.49633406058198626
1 0.49633406058198626
2 0.49633406058198626
3 0.49633406058198626
4 0.12087183801361165
doc:
0 0.25482305694621393
1 0.25482305694621393
3 0.25482305694621393
4 0.0620568558708622
5 0.8951785160431313
doc:
4 0.07979258234193365
6 0.5755093812740171
7 0.5755093812740171
8 0.5755093812740171
doc:
2 0.3485847413542797
4 0.08489056411237639
9 0.6122789185961829
10 0.3485847413542797
11 0.6122789185961829
doc:
10 0.37344696513776354
12 0.6559486886294514
13 0.6559486886294514


# ML에서 토픽 모델링(특정 단어의 비중) 을 할때 
<img src= 'resources/topicmodel.png'>

# Mobis_TextMining

In [17]:
# step 1 text download
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(categories = ["comp.graphics", "rec.sport.baseball", "sci.med"])

In [19]:
# setp 2 noun extract

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

tagged_list = [pos_tag(word_tokenize(doc))for doc in newsgroups.data]
nouns_list = [[t[0]for t in doc if t[1].startswith("N")] for doc in tagged_list]

In [20]:
# setp 3 표제어 extract
from nltk.stem import WordNetLemmatizer

lm = WordNetLemmatizer()

nouns_list = [[lm.lemmatize(w, pos = 'n') for w in doc] for doc in nouns_list]

In [21]:
# setp 4 stop-words remove
import re
token_list = [[text.lower() for text in doc] for doc in nouns_list]
token_list = [[re.sub(r"[^A-Za-z]+", '',word) for word in doc] for doc in token_list]

In [22]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words += ["", "subject", "article", "line", "year", "month", "address", "keyword", "msg"]

token_list = [[word for word in doc
              if (word not in stop_words) and (2 < len(word) < 10)]
             for doc in token_list]

In [24]:
# step 5 topic modeling
from gensim import corpora

dictionary = corpora.Dictionary(token_list)
doc_term_matrix = [dictionary.doc2bow(tokens) for tokens in token_list]

In [26]:
%%time
from gensim.models.ldamodel import LdaModel

model = LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics=3)

CPU times: user 7.93 s, sys: 40.1 ms, total: 7.97 s
Wall time: 2.54 s


In [27]:
model.print_topics()

[(0,
  '0.013*"lines" + 0.005*"game" + 0.005*"time" + 0.004*"geb" + 0.004*"banks" + 0.003*"people" + 0.003*"gordon" + 0.003*"day" + 0.003*"computer" + 0.003*"way"'),
 (1,
  '0.013*"lines" + 0.007*"image" + 0.006*"file" + 0.005*"team" + 0.005*"time" + 0.004*"game" + 0.004*"people" + 0.004*"program" + 0.004*"computer" + 0.004*"science"'),
 (2,
  '0.009*"lines" + 0.007*"image" + 0.005*"time" + 0.005*"point" + 0.005*"problem" + 0.004*"color" + 0.004*"program" + 0.003*"system" + 0.003*"anyone" + 0.003*"player"')]

In [32]:
# step viualization
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, doc_term_matrix, dictionary)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.004978 -0.025130       1        1  39.718594
2     -0.024503  0.016041       2        1  30.447409
0      0.029481  0.009089       3        1  29.833994, topic_info=         Term        Freq       Total Category  logprob  loglift
14        geb  218.000000  218.000000  Default  30.0000  30.0000
242     image  680.000000  680.000000  Default  29.0000  29.0000
3       banks  245.000000  245.000000  Default  28.0000  28.0000
15     gordon  188.000000  188.000000  Default  27.0000  27.0000
1073  michael  148.000000  148.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
116    number   89.917046  332.412018   Topic3  -6.0098  -0.0980
303     point   91.334778  359.901886   Topic3  -5.9942  -0.1618
0      anyone   92.080719  395.096771   Topic3  -5.9860  -0.2469
96       team   89.028122  388.054016   Topic3  -6.0197  -0.2627
235      file   92.202652  473.075806   Topic3  -5.9847  -0.4257

[286 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
6453       1  0.091372        abc
6453       2  0.822352        abc
6453       3  0.091372        abc
6366       1  0.894943     acsddc
13803      3  0.854002  aflatoxin
...      ...       ...        ...
1090       3  0.689509     yankee
2714       1  0.162791       yogi
2714       2  0.081395       yogi
2714       3  0.732558       yogi
2951       2  0.876771     zemcik

[507 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])